## Dataclass

In [3]:
from dataclasses import dataclass, field
from typing import List, Optional, Dict
from pydantic import BaseModel, Field, TypeAdapter

@dataclass
class User:
    id: int
    name: str = "JohnDoe"
    age: Optional[int] = field(
        default=None,
        metadata=dict(title="The age of the user", description="do not lie about your age", ge=10)
    )
    # Field work in normal dataclass with pydantic but not recommended just use BaseModel from pydantic
    height: Optional[float] = Field(None, title="The height in cm", ge=0.0, le=400.0)
    friends: List[int] = field(default_factory=lambda: [0])

In [4]:
print(TypeAdapter(User).json_schema())

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'JohnDoe', 'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'minimum': 10, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'do not lie about your age', 'title': 'The age of the user'}, 'height': {'anyOf': [{'maximum': 400.0, 'minimum': 0.0, 'type': 'number'}, {'type': 'null'}], 'default': None, 'title': 'The height in cm'}, 'friends': {'items': {'type': 'integer'}, 'title': 'Friends', 'type': 'array'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


## strict Model

In [5]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    username: str
    
user = User(id="1", username="JohnDoe") # not error because it will try to parseInt the id.
user

User(id=1, username='JohnDoe')

In [6]:
try:
    User.model_validate({"id": "1", "username": "JohnDoe"}, strict=True)
except Exception as e:
    print(e)

1 validation error for User
id
  Input should be a valid integer [type=int_type, input_value='1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_type


In [7]:
# !pip install pydantic-settings

## Pydantic Settings

In [8]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings

In [9]:
class Setting(BaseSettings):
    auth_key: str = Field(...)
    api_key: str = Field(...)
    
# print(Setting().model_dump())

In [10]:
import os 
from pydantic import AliasChoices

os.environ["AUTH_KEY"] = "auth_key_secret"
os.environ["API_KEY_2"] = "my_api_secretkey"

class Settings(BaseSettings):
    auth_key: str
    api_key: str = Field(validation_alias=AliasChoices("API_KEY_2", "API_KEY")) # choose from .env or here
    
settings = Settings()

print(settings)

auth_key='auth_key_secret' api_key='my_api_secretkey'
